In [1]:
import pandas as pd
import random
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

In [21]:
# Путь к файлу с данными по широте,долготе и тяжести льда.
path_excel = 'IntegrVelocity.xlsx'

## Загрузка данных с широтой и долготой пространственной сетки в полярной стереографической проекции

In [22]:
# Загрузка данных широты и долготы сетки. Каждая пара - левый вверхний угол участка 25х25 км.
Velocity_lon = pd.read_excel(path_excel, sheet_name='lon', header=None)
Velocity_lat = pd.read_excel(path_excel, sheet_name='lat', header=None)

In [30]:
fig, ax = plt.subplots(figsize=(36,24))
ax.plot(Velocity_lon.values.tolist(), Velocity_lat.values.tolist(), markersize=18,  color='blue', marker ='s', linestyle='None',zorder=1)
plt.title('Пространственная сетка в полярной стереографической проекции', fontsize=30)
plt.xlabel('Широта', fontsize=30)
plt.xticks(fontsize = 20)
plt.ylabel('Долгота', fontsize=30)
plt.yticks(fontsize = 20)
plt.show()

## Загрузка данных по тяжести льда для пространственной сетки

In [31]:
def plot_ice_map(path_excel, sheet_name, Velocity_lat, Velocity_lon):
    '''
        path_excel - путь к эксель файлу
        sheet_name - название листа чтения
        Velocity_lat - таблица pandas с широтой 
        Velocity_lon - таблица pandas с долготой

        На выходе изображение пространственной сетки с указанием тяжести льда на ней. Каждый цвет соответствует диапазону значений тяжести льда.
        Указаны в легенде на графике.
    '''
    Velocity_day = pd.read_excel(path_excel, sheet_name=sheet_name, header=None)
    color_df = Velocity_day.copy()
    
    color_df[Velocity_day < 0] = 'gray'
    color_df[(Velocity_day >= 0)&(Velocity_day <= 10)] = 'red'
    color_df[(Velocity_day > 10)&(Velocity_day <= 15)] = 'orange'
    color_df[(Velocity_day > 15)&(Velocity_day <= 19)] = 'yellow'
    color_df[Velocity_day > 19] = 'green'
    
    fig, ax = plt.subplots(figsize=(24,12))
    for i in range(Velocity_lat.shape[0]):
        for j in range(Velocity_lat.shape[1]):
            data_lat = Velocity_lat.iloc[i,j]
            data_lon = Velocity_lon.iloc[i,j]
            color_ = color_df.iloc[i,j]
            ax.plot(data_lon, data_lat, markersize=18,  color=color_, marker ='s', linestyle='None')
            
    plt.title(f'Пространственная сетка {sheet_name}', fontsize=30)
    gray_patch = mpatches.Patch(color='gray', label='Тяжесть льда < 0')
    red_patch = mpatches.Patch(color='red', label='0 <= Тяжесть льда < 10')
    orange_patch = mpatches.Patch(color='orange', label='10 < Тяжесть льда <= 15')
    yellow_patch = mpatches.Patch(color='yellow', label='15 < Тяжесть льда <= 19')
    green_patch = mpatches.Patch(color='green', label='Тяжесть льда > 19')
    plt.legend(handles=[gray_patch,red_patch, orange_patch,yellow_patch,green_patch],loc='lower left')
    plt.xlabel('Широта', fontsize=30)
    plt.xticks(fontsize = 20)
    plt.ylabel('Долгота', fontsize=30)
    plt.yticks(fontsize = 20)
    plt.show()

In [32]:
xls = pd.ExcelFile(path_excel)
print('Даты расчетов тяжести льда:',xls.sheet_names[2:])

In [33]:
plot_ice_map(path_excel, xls.sheet_names[2], Velocity_lat, Velocity_lon)

In [34]:
plot_ice_map(path_excel, xls.sheet_names[7], Velocity_lat, Velocity_lon)

In [52]:
plot_ice_map(path_excel, xls.sheet_names[-1], Velocity_lat, Velocity_lon)

Была построенна пространственная сетка в полярной стереографической проекции. Визуализировали значения тяжести льда на пространственной сетке. Каждому диапазону был присвоен свой цвет, который указан в легенде на графике.

## Загрузка данных графов

In [39]:
graph_points = pd.read_excel('ГрафДанные.xlsx', sheet_name='points')
graph_edges = pd.read_excel('ГрафДанные.xlsx', sheet_name='edges')

In [37]:
graph_points[['latitude','longitude']].min(), graph_points[['latitude','longitude']].max()

В данных по узлам графа присутствуют отрицательные значения, чтобы преобразовать в положительное значения необходимо сложить с 360.

In [43]:
# Отрицательное значение для бериногов пролив, переводим в положительное для отображения
graph_points.loc[graph_points[graph_points.longitude < 0].longitude.index, 'longitude'] = 360 + graph_points[graph_points.longitude < 0].longitude.values
# Номера гарфов узла для специального отображения на графике.
point_invert = [41,29,3,31,11,35,14,43,36]

In [45]:
# Координаты и названия для узлов графа
x = graph_points.longitude.values.tolist()
y = graph_points.latitude.values.tolist()
txt = graph_points.point_id.values.tolist()
name = graph_points.point_name.values.tolist()

fig, ax = plt.subplots(figsize=(36,24))
ax.plot(Velocity_lon.values.tolist(), Velocity_lat.values.tolist(), markersize=18,  color='blue', marker ='s', linestyle='None',zorder=1)
ax.scatter(x, y, c='g',zorder=2, linewidths=10)

for i, txt_ in enumerate(txt):
    if txt_ in point_invert:
        ax.annotate(txt_, (x[i], y[i]), xytext=(x[i]*1.01, y[i]*0.99), size=20, c='w')
        continue
    if (txt_ == 26) or (txt_ == 39):
        ax.annotate(txt_, (x[i], y[i]), xytext=(x[i]*0.985, y[i]), size=20, c='w')
        continue
    ax.annotate(txt_, (x[i], y[i]), xytext=(x[i]*0.99, y[i]*1.01), size=20, c='w')
# Построение ребер графа
for j in range(len(graph_edges)):
    start_point = graph_edges.iloc[j,:].start_point_id
    end_point = graph_edges.iloc[j,:].end_point_id
    start_data = graph_points[graph_points.point_id == start_point][['latitude','longitude']]
    x1 = start_data.longitude.values.tolist()
    y1 = start_data.latitude.values.tolist()
    end_data = graph_points[graph_points.point_id == end_point][['latitude','longitude']]
    x2 = end_data.longitude.values.tolist()
    y2 = end_data.latitude.values.tolist()
    ax.plot([x1, x2], [y1, y2], zorder=2, color='red', linewidth=6, alpha=0.5)

plt.text(210, 40, f'{graph_points.point_name.to_string()}', fontsize = 20)
plt.title('Пространственная сетка в полярной стереографической проекции c графом переходов', fontsize=30)
plt.xlabel('Широта', fontsize=30)
plt.xticks(fontsize = 20)
plt.ylabel('Долгота', fontsize=30)
plt.yticks(fontsize = 20)
plt.show()
plt.show()

Изображение каждого ребра графа, случайным цветом

In [46]:
# Координаты и названия для узлов графа
x = graph_points.longitude.values.tolist()
y = graph_points.latitude.values.tolist()
txt = graph_points.point_id.values.tolist()
name = graph_points.point_name.values.tolist()

fig, ax = plt.subplots(figsize=(36,24))
ax.plot(Velocity_lon.values.tolist(), Velocity_lat.values.tolist(), markersize=18,  color='blue', marker ='s', linestyle='None',zorder=1)
ax.scatter(x, y, c='g',zorder=2, linewidths=10)

for i, txt_ in enumerate(txt):
    if txt_ in point_invert:
        ax.annotate(txt_, (x[i], y[i]), xytext=(x[i]*1.01, y[i]*0.99), size=20, c='w')
        continue
    if (txt_ == 26) or (txt_ == 39):
        ax.annotate(txt_, (x[i], y[i]), xytext=(x[i]*0.985, y[i]), size=20, c='w')
        continue
    ax.annotate(txt_, (x[i], y[i]), xytext=(x[i]*0.99, y[i]*1.01), size=20, c='w')
# Построение ребер графа
for j in range(len(graph_edges)):
    start_point = graph_edges.iloc[j,:].start_point_id
    end_point = graph_edges.iloc[j,:].end_point_id
    start_data = graph_points[graph_points.point_id == start_point][['latitude','longitude']]
    x1 = start_data.longitude.values.tolist()
    y1 = start_data.latitude.values.tolist()
    end_data = graph_points[graph_points.point_id == end_point][['latitude','longitude']]
    x2 = end_data.longitude.values.tolist()
    y2 = end_data.latitude.values.tolist()
    color=(random.uniform(0, 1), random.uniform(0, 1), 0)
    ax.plot([x1, x2], [y1, y2], zorder=2, color=color, linewidth=6)

plt.text(210, 40, f'{graph_points.point_name.to_string()}', fontsize = 20)
plt.title('Пространственная сетка в полярной стереографической проекции c графом переходов', fontsize=30)
plt.xlabel('Широта', fontsize=30)
plt.xticks(fontsize = 20)
plt.ylabel('Долгота', fontsize=30)
plt.yticks(fontsize = 20)
plt.show()

Построенны данные графов, а именно узлы и ребра. По графикам видно, что узлы №26(мыс.Наглейнын) и №39(Рейд Певек) не имеют между собой общего ребра, но имеют ребро с узлом №18(Восточно-Сибирское - 1(восток)). 

## Загрузка данных расписания и местоположения ледоколов

In [11]:
shedudle = pd.read_excel('Расписание движения судов.xlsx')

shedudle_ships = shedudle.iloc[:42,:].sort_values(by=['Дата начала плавания']).reset_index(drop=True)
shedudle_icebreaker = shedudle.iloc[44:,2:].reset_index(drop=True)
shedudle_icebreaker.columns = shedudle_icebreaker.iloc[0,:]
shedudle_icebreaker = shedudle_icebreaker.iloc[1:,:].dropna()

Добавление широты и долготы для точек маршрута

In [12]:
data = []
for point in shedudle_icebreaker['Начальное положение ледоколов на 27 февраля 2022']:
    data_append = graph_points[graph_points.point_name.str.lower() == point.lower()][['latitude','longitude']].values[0]
    data.append(data_append)
shedudle_icebreaker[['latitude','longitude']] = data

data = []
for point in shedudle_ships['Пункт начала плавания']:
    data_append = graph_points[graph_points.point_name.str.lower() == point.lower()][['latitude','longitude']].values[0]
    data.append(data_append)
shedudle_ships[['latitude_start','longitude_start']] = data

data = []
for point in shedudle_ships['Пункт окончания плавания']:
    data_append = graph_points[graph_points.point_name.str.lower() == point.lower()][['latitude','longitude']].values[0]
    data.append(data_append)
shedudle_ships[['latitude_end','longitude_end']] = data

In [50]:
Velocity_day = pd.read_excel(path_excel, sheet_name=xls.sheet_names[2], header=None)
color_df = Velocity_day.copy()

color_df[Velocity_day < 0] = 'gray'
color_df[(Velocity_day >= 0)&(Velocity_day <= 10)] = 'red'
color_df[(Velocity_day > 10)&(Velocity_day <= 15)] = 'orange'
color_df[(Velocity_day > 15)&(Velocity_day <= 19)] = 'yellow'
color_df[Velocity_day > 19] = 'green'

fig, ax = plt.subplots(figsize=(36,24))
for i in range(Velocity_lat.shape[0]):
    for j in range(Velocity_lat.shape[1]):
        data_lat = Velocity_lat.iloc[i,j]
        data_lon = Velocity_lon.iloc[i,j]
        color_ = color_df.iloc[i,j]
        ax.plot(data_lon, data_lat, markersize=18,  color=color_, marker ='s', linestyle='None',zorder=1)
        
gray_patch = mpatches.Patch(color='gray', label='Тяжесть льда < 0')
red_patch = mpatches.Patch(color='red', label='0 <= Тяжесть льда < 10')
orange_patch = mpatches.Patch(color='orange', label='10 <= Тяжесть льда < 15')
yellow_patch = mpatches.Patch(color='yellow', label='15 <= Тяжесть льда < 19')
green_patch = mpatches.Patch(color='green', label='Тяжесть льда >= 19')
plt.legend(handles=[gray_patch,red_patch, orange_patch,yellow_patch,green_patch],loc='lower left', prop={'size': 20})

x = shedudle_icebreaker.longitude.values.tolist()
y = shedudle_icebreaker.latitude.values.tolist()
name = shedudle_icebreaker['Наименование'].values.tolist()

x_start = shedudle_ships.iloc[0,:]['longitude_start']
y_start = shedudle_ships.iloc[0,:]['latitude_start']
x_end = shedudle_ships.iloc[0,:]['longitude_end']
y_end = shedudle_ships.iloc[0,:]['latitude_end']


ax.scatter(x_start, y_start, c='b',zorder=2, linewidths=15)
ax.annotate('Старт', (x_start, y_start), xytext=(x_start*1.01, y_start*0.98), size=20, c='b')
ax.scatter(x_end, y_end, c='darkblue',zorder=2, linewidths=15)
ax.annotate('Конец', (x_end, y_end), xytext=(x_end*1.01, y_end*0.98), size=20, c='darkblue')

ax.scatter(x, y, c='purple',zorder=3, linewidths=10)
for i, txt_ in enumerate(name):
    ax.annotate(txt_, (x[i], y[i]), xytext=(x[i]*0.99, y[i]*1.01), size=25, c='purple')

plt.title('Расположение ледоколов и узлы маршрута судна ДЮК II на пространственной сетке со значениями тяжести льда', fontsize=30)
plt.xlabel('Широта', fontsize=30)
plt.xticks(fontsize = 20)
plt.ylabel('Долгота', fontsize=30)
plt.yticks(fontsize = 20)
plt.show()

In [51]:
Velocity_day = pd.read_excel(path_excel, sheet_name=xls.sheet_names[2], header=None)
color_df = Velocity_day.copy()

color_df[Velocity_day < 0] = 'gray'
color_df[(Velocity_day >= 0)&(Velocity_day <= 10)] = 'red'
color_df[(Velocity_day > 10)&(Velocity_day <= 15)] = 'orange'
color_df[(Velocity_day > 15)&(Velocity_day <= 19)] = 'yellow'
color_df[Velocity_day > 19] = 'green'

fig, ax = plt.subplots(figsize=(36,24))
for i in range(Velocity_lat.shape[0]):
    for j in range(Velocity_lat.shape[1]):
        data_lat = Velocity_lat.iloc[i,j]
        data_lon = Velocity_lon.iloc[i,j]
        color_ = color_df.iloc[i,j]
        ax.plot(data_lon, data_lat, markersize=18,  color=color_, marker ='s', linestyle='None',zorder=1)
        
gray_patch = mpatches.Patch(color='gray', label='Тяжесть льда < 0')
red_patch = mpatches.Patch(color='red', label='0 <= Тяжесть льда < 10')
orange_patch = mpatches.Patch(color='orange', label='10 <= Тяжесть льда < 15')
yellow_patch = mpatches.Patch(color='yellow', label='15 <= Тяжесть льда < 19')
green_patch = mpatches.Patch(color='green', label='Тяжесть льда >= 19')
plt.legend(handles=[gray_patch,red_patch, orange_patch,yellow_patch,green_patch],loc='lower left', prop={'size': 20})

x = shedudle_icebreaker.longitude.values.tolist()
y = shedudle_icebreaker.latitude.values.tolist()
name = shedudle_icebreaker['Наименование'].values.tolist()

x_start = shedudle_ships.iloc[0,:]['longitude_start']
y_start = shedudle_ships.iloc[0,:]['latitude_start']
x_end = shedudle_ships.iloc[0,:]['longitude_end']
y_end = shedudle_ships.iloc[0,:]['latitude_end']


ax.scatter(x_start, y_start, c='b',zorder=2, linewidths=15)
ax.annotate('Старт', (x_start, y_start), xytext=(x_start*1.01, y_start*0.98), size=20, c='b')
ax.scatter(x_end, y_end, c='darkblue',zorder=2, linewidths=15)
ax.annotate('Конец', (x_end, y_end), xytext=(x_end*1.01, y_end*0.98), size=20, c='darkblue')

ax.scatter(x, y, c='purple',zorder=3, linewidths=10)
for i, txt_ in enumerate(name):
    ax.annotate(txt_, (x[i], y[i]), xytext=(x[i]*0.99, y[i]*1.01), size=25, c='purple')

# Построение ребер графа
for j in range(len(graph_edges)):
    start_point = graph_edges.iloc[j,:].start_point_id
    end_point = graph_edges.iloc[j,:].end_point_id
    start_data = graph_points[graph_points.point_id == start_point][['latitude','longitude']]
    x1 = start_data.longitude.values.tolist()
    y1 = start_data.latitude.values.tolist()
    end_data = graph_points[graph_points.point_id == end_point][['latitude','longitude']]
    x2 = end_data.longitude.values.tolist()
    y2 = end_data.latitude.values.tolist()
    ax.plot([x1, x2], [y1, y2], zorder=2, color='k', linewidth=3, alpha=0.4)


plt.title('Расположение ледоколов узлы маршрута судна ДЮК II и ребра графов на пространственной сетке со значениями тяжести льда', fontsize=30)
plt.xlabel('Широта', fontsize=30)
plt.xticks(fontsize = 20)
plt.ylabel('Долгота', fontsize=30)
plt.yticks(fontsize = 20)
plt.show()

## Заключение

1. Построена пространственная сетка в стереографической проекции. <br>
2. Построены примеры пространственных сеток с отображением значений тяжести льда в разные периоды времени. <br>
3. Построены узлы и ребра графа вероятных маршрутов взятых из исторических данных. <br>
4. Построены положения ледоколов, начальная и конечная точка маршрута для первого корабля из заявок ДЮК II.<br>
При анализе данных обнаружилась отрицательное значение положения для узла Берингов пролив. При анализе пространственной карты с отображением тяжести льда было замечено, что узлы №1(Дудинка),5(Окно в Европу),29(Архангельск),35(Терминал Утренний),38(Кромка льда на Востоке),46(Окно в Азию) находятся в зонах, где тяжесть льда отрицательная.